In [ ]:
from src.embedor import *
from src.data.data import *
from src.plotting import *
from src.models.mlp import *
%load_ext autoreload

In [ ]:
X, mnist_labels = get_mnist_data(n_samples=2000)

In [ ]:
embedor = EmbedORFast()
emb = embedor.fit_transform(X)
plot_graph_2D(emb, embedor.G, edge_width=0, node_color=mnist_labels)

In [ ]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_dim = X.shape[1]

device = torch.device("cpu")
# nn = MLP(
#     input_dim,
#     hidden_dim=50,
#     output_dim=2,
#     num_layers=2).to(device)

emb = torch.nn.Parameter(torch.tensor(embedor.spectral_init, dtype=torch.float, requires_grad=True).to(device))

subsample_indices = torch.tensor(embedor.subsample_indices).to(device)
epochs_per_pair_positive = torch.tensor(embedor.epochs_per_pair_positive).to(device)
epochs_per_pair_negative = torch.tensor(embedor.epochs_per_pair_negative).to(device)

X = torch.tensor(X, dtype=torch.float).to(device)

optimizer = torch.optim.Adam([emb], lr=1e-2)
# nn.train()
n_epochs = embedor.epochs
next_epoch_positive = torch.tensor(epochs_per_pair_positive).to(device)
next_epoch_negative = torch.tensor(epochs_per_pair_negative).to(device)
losses = []
for iter in range(250):
    optimizer.zero_grad()
    # get pairs for this epoch
    idx_positive = torch.where(next_epoch_positive <= iter)[0]
    idx_negative = torch.where(next_epoch_negative <= iter)[0]
    if len(idx_positive) == 0 and len(idx_negative) == 0:
        continue
    # update next epoch
    next_epoch_positive[idx_positive] += epochs_per_pair_positive[idx_positive]
    next_epoch_negative[idx_negative] += epochs_per_pair_negative[idx_negative]
    # get pairs
    pairs_positive = subsample_indices[:, idx_positive]
    pairs_negative = subsample_indices[:, idx_negative]
    # stack to get batch
    # src_embeddings_positive = nn(X[pairs_positive[0]])
    # dst_embeddings_positive = nn(X[pairs_positive[1]])
    # src_embeddings_negative = nn(X[pairs_negative[0]])
    # dst_embeddings_negative = nn(X[pairs_negative[1]])
    src_embeddings_positive = emb[pairs_positive[0]]
    dst_embeddings_positive = emb[pairs_positive[1]]
    src_embeddings_negative = emb[pairs_negative[0]]
    dst_embeddings_negative = emb[pairs_negative[1]]

    f_positive = torch.pow(1 + torch.norm(src_embeddings_positive - dst_embeddings_positive, dim=1, p=2), -1)
    f_negative = torch.pow(1 + torch.norm(src_embeddings_negative - dst_embeddings_negative, dim=1, p=2), -1)


    ## TODO: make sure no pairs (i,j) are sampled s.t. i == j!! 
    loss_positive = -torch.log(f_positive)
    loss_negative = -torch.log(1 - f_negative) * embedor.gamma
    loss = loss_positive.sum() + loss_negative.sum()

    loss.backward()
    optimizer.step()
    print(f"iter:{iter}, loss:{loss.item()}")
    losses.append(loss.item())    


In [ ]:
final_embeddings = emb
plot_graph_2D(final_embeddings.detach().cpu().numpy(), embedor.G, edge_width=0, node_color=mnist_labels)

In [ ]:
plt.plot(losses)
plt.xlabel("Epoch")
plt.ylabel("Loss")